Script Description
This script loads a pre-processed dataset, converts the units of NEE variables as weel as computes the 

File Name: 04_01_Computing_the_nightime_NEE.ipynb

Date: 2024

Created by: Rob Alamgir

Version: 1.0

References:

#### Import the relevant packages

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

##### Check the presence of files in a specific directory & Extract only specific columns from all the CSV files in a specific folder

In [2]:
directory_path = 'C:/Data_MSc_Thesis/EC_Tower_Data/Friesland_EC_Tower_Data'   # Specify the directory path
files = os.listdir(directory_path)                                            # Get a list of all files and directories in the specified directory

files = [f for f in files if os.path.isfile(os.path.join(directory_path, f))] # Filter out directories and only list files
files_with_data = []                                                          # Initialize lists to store files with data and without data
files_without_data = []
data_list = []                                                                # Initialize an empty list to store the data

# Loop through each file
for file in files:
    file_path = os.path.join(directory_path, file)                            # Read the CSV file
    df = pd.read_csv(file_path, low_memory=False)
    # Check if the required columns exist and contain data
    if {'datetime','daytime','NEE_CO2','NEE_CH4'}.issubset(df.columns):
        # Check if there is any non-null data in the specified columns
        if df[['datetime','daytime','NEE_CO2','NEE_CH4']].notnull().any().any():
            # If there's data, add the file to the data list and to files_with_data
            df_filtered = df[['datetime','daytime','NEE_CO2','NEE_CH4']].copy()
            df_filtered['location'] = file                                    # Add the file name as a new column
            data_list.append(df_filtered)
            files_with_data.append(file)
        else:           
            files_without_data.append(file)                                  # If no data, add the file to files_without_data
    else:        
        files_without_data.append(file)                                      # If the required columns are missing
if data_list:
    final_df = pd.concat(data_list, ignore_index=True)                       # Combine all the data into a single DataFrame, if there is any data
    print("Data sucessfully Extracted & ")
else:
    print("No data available to merge.")
print("\nFiles with data:")                                                 # Print the files with and without data
print(files_with_data)
print("\nFiles without data:")
print(files_without_data)

Data sucessfully Extracted & 

Files with data:
['ALB_MS.csv', 'ALB_RF.csv', 'AMM.csv', 'AMR.csv', 'BUO.csv', 'BUW.csv', 'HOC.csv', 'HOH.csv', 'LDC.csv', 'LDH.csv']

Files without data:
[]


In [ ]:
#final_df.head(5)
#final_df.tail(5)
#final_df.dtypes
#final_df['daytime'].isna().sum()

In [3]:
final_df = final_df.drop(index=0)                            # Remove the first row (index 0)
final_df['datetime'] = pd.to_datetime(final_df['datetime'], 
                                      errors='coerce')       # Convert 'datetime' column to datetime type
final_df['date'] = final_df['datetime'].dt.date              # Convert datetime to a date column
final_df['date'] = pd.to_datetime(final_df['date'],          # Convert 'date' column to date type
                                  format= '%Y-%m-%d', errors='coerce')

final_df['daytime'] = pd.to_numeric(final_df['daytime'], errors='coerce') # Convert columns to numeric types
final_df['NEE_CO2'] = pd.to_numeric(final_df['NEE_CO2'], errors='coerce') # Convert columns to numeric types
final_df['NEE_CH4'] = pd.to_numeric(final_df['NEE_CH4'], errors='coerce') # Convert columns to numeric types
final_df['location'] = final_df['location'].astype('category')                    # Convert 'location' to categorical type (if it has repeating values)
final_df['location'] = final_df['location'].str.replace('.csv', '', regex=False)  # Remove the '.csv' from all entries in the 'location' column
final_df.rename(columns={'date': 'Date', 'location': 'Source'}, inplace=True)     # Rename the columns

# Group by location and date, then calculate the mean for relevant columns
Wholeday_df = final_df[final_df['daytime'].isin([0, 1])]     # Extract rows where 'daytime' is 0 or 1
Wholeday_df['date'] = Wholeday_df['datetime'].dt.date        # Convert datetime to a date column
Wholeday_df.loc[:, 'date'] = Wholeday_df['datetime'].dt.date # Convert datetime to a date column safely using .loc
Wholeday_daily_means = (Wholeday_df.groupby(['Source', 'date'], as_index=False)
                          .mean(numeric_only=True))          # Ensures non-numeric columns like 'location' are excluded

nighttime_df = final_df[final_df['daytime'] == 0]              # Extract rows where 'daytime' is 0
nighttime_df['date'] = nighttime_df['datetime'].dt.date        # Convert datetime to a date column
nighttime_df.loc[:, 'date'] = nighttime_df['datetime'].dt.date # Convert datetime to a date column safely using .loc
nighttime_daily_means = (nighttime_df.groupby(['Source', 'Date'], as_index=False)
                         .mean(numeric_only=True))             # Ensures non-numeric columns like 'location' are excluded

daytime_df = final_df[final_df['daytime'] == 1]                # Extract rows where 'daytime' is 1
daytime_df['date'] = daytime_df['datetime'].dt.date            # Convert datetime to a date column
daytime_df.loc[:, 'date'] = daytime_df['datetime'].dt.date     # Convert datetime to a date column safely using .loc
daytime_daily_means = (daytime_df.groupby(['Source', 'Date'], as_index=False)
                         .mean(numeric_only=True))             # Ensures non-numeric columns like 'location' are excluded

NaN_df = final_df[final_df['daytime'].isna()]      # Extract rows where 'daytime' is NaN
NaN_df['date'] = NaN_df['datetime'].dt.date        # Convert datetime to a date column
NaN_df.loc[:, 'date'] = NaN_df['datetime'].dt.date # Convert datetime to a date column safely using .loc
NaN_daily_means = (NaN_df.groupby(['Source', 'Date'], as_index=False)
                         .mean(numeric_only=True))  # Ensures non-numeric columns like 'location' are excluded

CO2_conversion_factor = 38.016  # Conversion factor for CO₂
CH4_conversion_factor = 13.824  # Conversion factor for CH4

# Explicitly convert each column and add new columns
Wholeday_daily_means['NEE_CO2_kg_day_ha'] = Wholeday_daily_means['NEE_CO2'] * CO2_conversion_factor
Wholeday_daily_means['NEE_CH4_kg_day_ha'] = Wholeday_daily_means['NEE_CH4'] * CH4_conversion_factor
nighttime_daily_means['NEE_CO2_kg_day_ha'] = nighttime_daily_means['NEE_CO2'] * CO2_conversion_factor
nighttime_daily_means['NEE_CH4_kg_day_ha'] = nighttime_daily_means['NEE_CH4'] * CH4_conversion_factor
daytime_daily_means['NEE_CO2_kg_day_ha'] = daytime_daily_means['NEE_CO2'] * CO2_conversion_factor
daytime_daily_means['NEE_CH4_kg_day_ha'] = daytime_daily_means['NEE_CH4'] * CH4_conversion_factor
NaN_daily_means['NEE_CO2_kg_day_ha'] = NaN_daily_means['NEE_CO2'] * CO2_conversion_factor
NaN_daily_means['NEE_CH4_kg_day_ha'] = NaN_daily_means['NEE_CH4'] * CH4_conversion_factor

# Group by Source (location) and week, then calculate the mean for relevant columns
Wholeday_weekly_means = (Wholeday_df.set_index('datetime')    # Set datetime as the index
                          .groupby('Source')                  # Group by location (Source)
                          .resample('W')                      # Resample by week
                          .mean(numeric_only=True)            # Calculate mean for numeric columns
                          .reset_index())                     # Reset index to have 'datetime' as a column

nighttime_weekly_means = (nighttime_df.set_index('datetime')  
                          .groupby('Source')                  
                          .resample('W')                      
                          .mean(numeric_only=True)            
                          .reset_index())                     

daytime_weekly_means = (daytime_df.set_index('datetime')  
                        .groupby('Source')  
                        .resample('W')  
                        .mean(numeric_only=True) 
                        .reset_index())  

NaN_weekly_means = (NaN_df.set_index('datetime')  
                    .groupby('Source')  
                    .resample('W')  
                    .mean(numeric_only=True)  
                    .reset_index()) 

# Explicitly convert each column and add new columns
Wholeday_weekly_means['NEE_CO2_kg_day_ha'] = Wholeday_weekly_means['NEE_CO2'] * CO2_conversion_factor
Wholeday_weekly_means['NEE_CH4_kg_day_ha'] = Wholeday_weekly_means['NEE_CH4'] * CH4_conversion_factor
nighttime_weekly_means['NEE_CO2_kg_day_ha'] = nighttime_weekly_means['NEE_CO2'] * CO2_conversion_factor
nighttime_weekly_means['NEE_CH4_kg_day_ha'] = nighttime_weekly_means['NEE_CH4'] * CH4_conversion_factor
daytime_weekly_means['NEE_CO2_kg_day_ha'] = daytime_weekly_means['NEE_CO2'] * CO2_conversion_factor
daytime_weekly_means['NEE_CH4_kg_day_ha'] = daytime_weekly_means['NEE_CH4'] * CH4_conversion_factor
NaN_weekly_means['NEE_CO2_kg_day_ha'] = NaN_weekly_means['NEE_CO2'] * CO2_conversion_factor
NaN_weekly_means['NEE_CH4_kg_day_ha'] = NaN_weekly_means['NEE_CH4'] * CH4_conversion_factor

C:\Users\robdu\AppData\Local\Temp\ipykernel_26056\727138151.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Wholeday_df['date'] = Wholeday_df['datetime'].dt.date        # Convert datetime to a date column
C:\Users\robdu\AppData\Local\Temp\ipykernel_26056\727138151.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nighttime_df['date'] = nighttime_df['datetime'].dt.date        # Convert datetime to a date column
C:\Users\robdu\AppData\Local\Temp\ipykernel_26056\727138151.py:29: SettingWithCopyWarning:

In [4]:
#Load and preprocess data
data_path = "C:/Data_MSc_Thesis/Pre_Processed_Data/Pre_Processed_Data_All_Locations_Updated_4.csv"
complete_dataset = pd.read_csv(data_path)
complete_dataset['Date'] = pd.to_datetime(complete_dataset['Date'], format='%Y-%m-%d')
complete_dataset = complete_dataset[complete_dataset['Date'] >= '2020-01-01'] # Filter for dates greater than or equal to January 1, 2020
#complete_dataset['SWCT_1_015'] = complete_dataset['SWCT_1_015'] / 100  # Scale the SWCT_1_015 column

#print(complete_dataset.head(10))
#print(complete_dataset.describe())
#print(complete_dataset.columns)
#print(complete_dataset.dtypes) # Check the structure of all columns (data types)

In [5]:
# Adding the suffixes to the specified columns 
DAv_24hrs_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_DAv_24hrs',
                               'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_DAv_24hrs',
                               'date':'Date'} 
Wholeday_daily_means.rename(columns=DAv_24hrs_columns_to_rename, inplace=True)
Wholeday_daily_means['Date'] = pd.to_datetime(Wholeday_daily_means['Date'])   

DAv_NT_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_DAv_NT',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_DAv_NT'}
nighttime_daily_means.rename(columns=DAv_NT_columns_to_rename, inplace=True)

DAv_DT_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_DAv_DT',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_DAv_DT'}
daytime_daily_means.rename(columns=DAv_DT_columns_to_rename, inplace=True)

DAv_NaN_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_DAv_NaN',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_DAv_NaN'}
NaN_daily_means.rename(columns=DAv_NaN_columns_to_rename, inplace=True)

# Adding the suffixes to the specified columns 
WAv_24hrs_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_WAv_24hrs',
                               'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_WAv_24hrs',
                               'datetime':'Date'} 
Wholeday_weekly_means.rename(columns=WAv_24hrs_columns_to_rename, inplace=True)

WAv_NT_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_WAv_NT',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_WAv_NT',
                            'datetime':'Date'}
nighttime_weekly_means.rename(columns=WAv_NT_columns_to_rename, inplace=True)

WAv_DT_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_WAv_DT',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_WAv_DT',
                            'datetime':'Date'}
daytime_weekly_means.rename(columns=WAv_DT_columns_to_rename, inplace=True)

WAv_NaN_columns_to_rename = {'NEE_CO2_kg_day_ha': 'NEE_CO2_kg_day_ha_WAv_NaN',
                            'NEE_CH4_kg_day_ha': 'NEE_CH4_kg_day_ha_WAv_NaN',
                             'datetime':'Date'}
NaN_weekly_means.rename(columns=WAv_NaN_columns_to_rename, inplace=True)

# Merge 'complete_dataset' with 'nighttime_daily_means' based on 'Source' and 'Date'
complete_dataset = complete_dataset.merge(
    Wholeday_daily_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_DAv_24hrs', 'NEE_CH4_kg_day_ha_DAv_24hrs']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    nighttime_daily_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_DAv_NT', 'NEE_CH4_kg_day_ha_DAv_NT']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    daytime_daily_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_DAv_DT', 'NEE_CH4_kg_day_ha_DAv_DT']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    NaN_daily_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_DAv_NaN', 'NEE_CH4_kg_day_ha_DAv_NaN']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    Wholeday_weekly_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_WAv_24hrs', 'NEE_CH4_kg_day_ha_WAv_24hrs']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    nighttime_weekly_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_WAv_NT', 'NEE_CH4_kg_day_ha_WAv_NT']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    daytime_weekly_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_WAv_DT', 'NEE_CH4_kg_day_ha_WAv_DT']], 
    on=['Source', 'Date'], how='left')

complete_dataset = complete_dataset.merge(
    NaN_weekly_means[['Source', 'Date', 'NEE_CO2_kg_day_ha_WAv_NaN', 'NEE_CH4_kg_day_ha_WAv_NaN']], 
    on=['Source', 'Date'], how='left')

In [6]:
#print(complete_dataset.head(10))
#print(complete_dataset.tail(10))
#print(complete_dataset.describe())
print(complete_dataset.columns)
#print(complete_dataset.dtypes) # Check the structure of all columns (data types)

Index(['Date', 'Source', 'S1_VSM', 'S1_Backscatter', 'Planet_SWC',
       'Available_soil_storage_mm', 'S2_NDVI', 'S2_EVI', 'S2_NDMI', 'L8_9_LST',
       ...
       'NEE_CO2_kg_day_ha_DAv_NaN', 'NEE_CH4_kg_day_ha_DAv_NaN',
       'NEE_CO2_kg_day_ha_WAv_24hrs', 'NEE_CH4_kg_day_ha_WAv_24hrs',
       'NEE_CO2_kg_day_ha_WAv_NT', 'NEE_CH4_kg_day_ha_WAv_NT',
       'NEE_CO2_kg_day_ha_WAv_DT', 'NEE_CH4_kg_day_ha_WAv_DT',
       'NEE_CO2_kg_day_ha_WAv_NaN', 'NEE_CH4_kg_day_ha_WAv_NaN'],
      dtype='object', length=106)


In [7]:
NEE_Columns = ['NEE_CO2_kg_day_ha_DAv_24hrs', 'NEE_CH4_kg_day_ha_DAv_24hrs',
       'NEE_CO2_kg_day_ha_DAv_NT', 'NEE_CH4_kg_day_ha_DAv_NT',
       'NEE_CO2_kg_day_ha_DAv_DT', 'NEE_CH4_kg_day_ha_DAv_DT',
       'NEE_CO2_kg_day_ha_DAv_NaN', 'NEE_CH4_kg_day_ha_DAv_NaN',
       'NEE_CO2_kg_day_ha_WAv_24hrs', 'NEE_CH4_kg_day_ha_WAv_24hrs',
       'NEE_CO2_kg_day_ha_WAv_NT', 'NEE_CH4_kg_day_ha_WAv_NT',
       'NEE_CO2_kg_day_ha_WAv_DT', 'NEE_CH4_kg_day_ha_WAv_DT',
       'NEE_CO2_kg_day_ha_WAv_NaN', 'NEE_CH4_kg_day_ha_WAv_NaN']

complete_dataset[NEE_Columns].describe()

,NEE_CO2_kg_day_ha_DAv_24hrs,NEE_CH4_kg_day_ha_DAv_24hrs,NEE_CO2_kg_day_ha_DAv_NT,NEE_CH4_kg_day_ha_DAv_NT,NEE_CO2_kg_day_ha_DAv_DT,NEE_CH4_kg_day_ha_DAv_DT,NEE_CO2_kg_day_ha_DAv_NaN,NEE_CH4_kg_day_ha_DAv_NaN,NEE_CO2_kg_day_ha_WAv_24hrs,NEE_CH4_kg_day_ha_WAv_24hrs,NEE_CO2_kg_day_ha_WAv_NT,NEE_CH4_kg_day_ha_WAv_NT,NEE_CO2_kg_day_ha_WAv_DT,NEE_CH4_kg_day_ha_WAv_DT,NEE_CO2_kg_day_ha_WAv_NaN,NEE_CH4_kg_day_ha_WAv_NaN
count,3025.000000,638.000000,2699.000000,564.000000,2851.000000,605.000000,0.0,0.0,638.000000,164.000000,612.000000,154.000000,634.000000,160.000000,0.0,0.0
mean,-49.782099,0.152031,228.685678,0.224932,-189.443748,0.115737,NaN,NaN,-59.732834,0.149712,240.281903,0.232670,-201.517361,0.112851,NaN,NaN
std,209.624127,0.373912,202.275079,0.465260,225.336944,0.404873,NaN,NaN,156.324916,0.244365,163.174290,0.465885,177.069689,0.257871,NaN,NaN
min,-1675.873014,-1.867739,-1746.050930,-1.076622,-1382.717192,-4.303245,NaN,NaN,-1011.771510,-0.296423,-225.404498,-0.572024,-1011.771510,-0.715384,NaN,NaN
25%,-161.152703,0.012103,106.794807,0.053905,-331.900859,-0.007516,NaN,NaN,-148.667964,0.035118,113.606590,0.072223,-320.418380,0.005296,NaN,NaN
50%,-7.204201,0.088776,196.171017,0.143938,-169.444511,0.063117,NaN,NaN,-16.867742,0.096731,206.772413,0.136230,-177.898068,0.082512,NaN,NaN
75%,66.859042,0.210435,343.232444,0.289141,-39.539710,0.180575,NaN,NaN,43.544255,0.196542,342.480872,0.293980,-70.477810,0.173908,NaN,NaN
max,1872.233637,5.630225,1872.233637,4.416509,1723.185827,6.272818,NaN,NaN,841.213562,1.946210,1872.233637,4.411077,206.880964,2.294616,NaN,NaN


In [ ]:
Air_Temperature_Columns = ['ATMP', 'ATMP_f', 'TA_KNMI_270']

complete_dataset[Air_Temperature_Columns].describe()

#### Export Updated Dataset

In [8]:
#export_data_path = "C:/Data_MSc_Thesis/Pre_Processed_Data/Pre_Processed_Data_All_Locations_Updated_5.csv"
#complete_dataset.to_csv(export_data_path, index=False)
#print(f" dataset exported to {export_data_path}.")

 dataset exported to C:/Data_MSc_Thesis/Pre_Processed_Data/Pre_Processed_Data_All_Locations_Updated_5.csv.
